# AML — Task 1
## Predict the age of a brain from MRI features
---

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from pandas_profiling import ProfileReport

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, f_regression, mutual_info_regression, SelectKBest
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.neighbors import LocalOutlierFactor

In [3]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

## Import datasets

In [4]:
def import_data(folder="data/", extension=""):
    X_train = pd.read_csv(folder + 'X_train' + extension + '.csv').drop(columns=['id'])
    y_train = pd.read_csv(folder + 'y_train' + extension + '.csv').drop(columns=['id'])
    X_test = pd.read_csv(folder + 'X_test' + extension + '.csv').drop(columns=['id'])
    return X_train, y_train, X_test

---
## Exporting Data as csv

In [5]:
def export_to_csv(X_train_cleaned, y_train_cleaned, X_test_cleaned, folder="data/"):
    X_train_cleaned.to_csv(folder + 'X_train_cleaned.csv', index=False)
    y_train_cleaned.to_csv(folder + 'y_train_cleaned.csv', index=False)
    X_test_cleaned.to_csv(folder + 'X_test_cleaned.csv', index=False)

---
## Task 1: Outlier detection

In [6]:
"""Remove the ouliers from our dataset. Replace temporarily the Nan values by the mean to perform outlier selection

Parameters
----------
X_train : pd.df
    The features (what we will use to see the outliers)
y_train : pd.df
    The labels
contamination : int, optional
    The percent of outliers found by the isolation forest if it's used.
verbose : int, optional
    If not set to 0, print messages
    
Return
------
(pd.df, pd.df)
    The data with the outliers rows removed
"""
def remove_outliers(X_train, y_train, contamination='auto', verbose=1, method="LocalOutlierFactor"):
    # Save a mask of the imputed values to be able to redo the imputation once the outlier detection is done
    X_train_null_mask = X_train.isna()
    
    # Need to impute nan values for the outlier detection to work (cannot deal with nan)
    X_train_imputed = pd.DataFrame(SimpleImputer(strategy="median", verbose=verbose).fit_transform(X_train))
    
    clf = None
    if method=="LocalOutlierFactor":
        clf = LocalOutlierFactor(contamination=contamination)
    elif method=="IsolationForest":
        clf = IsolationForest(contamination=contamination, random_state=0, verbose=verbose)
    else:
        raise AttributeError(f"Unvalid argument for method, must be 'LocalOutlierFactor' or 'IsolationForest', not '{method}'")
        
    outliers_mask = pd.Series(clf.fit_predict(X_train_imputed)).map({1:1, -1:0}) #Mask with 0 for outliers and 1 for non outliers
    
    if verbose:
        print(f"Detected {(outliers_mask == 0).sum()} outliers with method {method}, out of {outliers_mask.shape[0]} samples ({100 * (outliers_mask == 0).sum() / outliers_mask.shape[0]:.2f}%).")
    
    #Replace the Nan values (The outlier detection shouldn't replace NaN values by itself)
    X_train = pd.DataFrame(X_train).mask(X_train_null_mask, other=np.NaN, inplace=False)
    
    # Remove outliers from the training set
    X_train = np.array(X_train)[outliers_mask == 1, :]
    y_train = np.array(y_train)[outliers_mask == 1, :]
    
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)
    
    return (X_train, y_train)

---
## Task 2: Data scaling
Done as soon as possible because can have an effect (e.g. on distances for `KNNImputer`)

In [7]:
def scale(X_train_no_outliers, X_test):
    # Do the scaling, saving the scaler to use it for X_test too. No need imputation, just ignore Nan values
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train_no_outliers))
    # Cast X_test to np.array to avoid warning of model trained without feature names but X having some.
    X_test_scaled = pd.DataFrame(scaler.transform(np.array(X_test)))
    return (X_train_scaled, X_test_scaled)

---
## Task 3: impute values

In [89]:
def impute_values(X_train, X_test, method='knn', verbose=2, max_iter=15):
    if verbose:
        print(f"For the train dataset, there are {np.array(X_train.isna()).sum().sum()} nan values, out of {X_train.shape[0]*X_train.shape[1]} ({100*np.array(X_train.isna()).sum().sum()/(X_train.shape[0]*X_train.shape[1]):.2f}%).")
    
    imputer = None
    if method == 'knn':
        imputer = KNNImputer(n_neighbors=6, weights='uniform').fit(X_train)
    elif method == 'iterative':
        # Runs VERY slowly and might cause the kernel to crash due to intenisve use of RAM
        imputer = IterativeImputer(random_state=0, max_iter=max_iter, verbose=verbose, initial_strategy='median').fit(X_train)
    else:
        raise AttributeError(f"Unvalid argument for method, must be 'knn' or 'iterative', not '{method}'")
    
    X_train_imputed = pd.DataFrame(imputer.transform(X_train))
    X_test_imputed = pd.DataFrame(imputer.transform(X_test))
    return (X_train_imputed, X_test_imputed)

---
## Task 4: Feature selection

In [9]:
def select_features(X_train_imputed, X_test_imputed):
    X_train_selected_features, X_test_selected_features = remove_constant_features(X_train_imputed, X_test_imputed)
    X_train_selected_features, X_test_selected_features = remove_too_coorelated_features(X_train_selected_features, X_test_selected_features)
    X_train_selected_features, X_test_selected_features = remove_random_features(X_train_selected_features, y_train, X_test_selected_features, percentile=80)

    return X_train_selected_features, X_test_selected_features

In [10]:
def remove_constant_features(X_train_imputed, X_test_imputed, verbose=1):
    X_train_selected_features = X_train_imputed.loc[:, np.logical_and(X_train_imputed != X_train_imputed.iloc[0], X_train_imputed.notna()).any()]
    X_test_selected_features = X_test_imputed.loc[:, np.logical_and(X_train_imputed != X_train_imputed.iloc[0], X_train_imputed.notna()).any()]
    
    if verbose:
        print(f"{X_train_imputed.shape[1]-X_train_selected_features.shape[1]} features removed because of constant values ({100*(X_train_imputed.shape[1]-X_train_selected_features.shape[1])/X_train_imputed.shape[1]:.2f}%)")
    
    return X_train_selected_features, X_test_selected_features

In [11]:
def remove_too_coorelated_features(X_train, X_test, threshold=0.7, verbose=1):
    X_train_corr_ = X_train.corr()

    X_train_too_correlated = (X_train_corr_.mask(
        np.tril(np.ones([len(X_train_corr_)]*2, dtype=bool))).abs() > threshold).any()
    
    
    X_train_selected_features = X_train.loc[:, (~X_train_too_correlated)]
    X_test_selected_features = X_test.loc[:, (~X_train_too_correlated)]
    
    if verbose:
        print(f"{X_train.shape[1]-X_train_selected_features.shape[1]} features removed because of correlation > {threshold} ({100*(X_train.shape[1]-X_train_selected_features.shape[1])/X_train.shape[1]:.2f}%)")

    return X_train_selected_features, X_test_selected_features

In [12]:
def remove_random_features(X_train, y_train, X_test, Xtrm=None, Xtem=None, k=190, verbose=1):
    selector = SelectKBest(f_regression, k=k) # modify here
    selector.fit(X_train, np.array(y_train).ravel())
    X_train_selected_features = pd.DataFrame(selector.transform(X_train))
    X_test_selected_features = pd.DataFrame(selector.transform(X_test))
    if Xtrm is not None:
        Xtrm = pd.DataFrame(selector.transform(Xtrm))
    if Xtem is not None:
        Xtem = pd.DataFrame(selector.transform(Xtem))
    
    if verbose:
        print(f"{X_train.shape[1]-X_train_selected_features.shape[1]} features removed because of low correlation with target ({100*(X_train.shape[1]-X_train_selected_features.shape[1])/X_train.shape[1]:.2f}%).")
        
    return X_train_selected_features, X_test_selected_features, Xtrm, Xtem

---
## Task 5 Models

In [13]:
def model(X_train_cleaned, y_train_cleaned):
    gs_lasso = best_lasso(X_train_cleaned, y_train_cleaned)
    gs_svr = best_svr(X_train_cleaned, y_train_cleaned)
    gs_gbr = best_gbr(X_train_cleaned, y_train_cleaned)
    gs_ada = best_Adaboost(X_train, y_train)
    gs_rforest = best_random_forest(X_train, y_train)
    max_score = max(max(max(max(gs_gbr.best_score_, gs_lasso.best_score_), gs_svr.best_score_), gs_ada.best_score_), gs_rforest.best_score)

    return gs_lasso if gs_lasso.best_score_ == max_score else gs_svr if gs_svr.best_score_ == max_score else gs_ada if gs_ada.best_score_ == max_score else gs_rforest if gs_rforest.best_score == max_score else gs_gbr

### Model 1: Lasso

In [14]:
def best_lasso(X_train_cleaned, y_train_cleaned):
    lasso = Lasso(max_iter=100000)
    gs_lasso_params = {
    'alpha': np.logspace(-1, 0, 20),
    }
    gs_lasso = GridSearchCV(lasso, gs_lasso_params, cv=5, verbose=3)
    gs_lasso.fit(X_train_cleaned, y_train_cleaned)
    print(f"The best validation score obtained is {gs_lasso.best_score_:.5f} with\n\talpha: {gs_lasso.best_params_['alpha']}")
    return gs_lasso;

### Model 2: SVR (SVM for regression)

In [165]:
def best_svr(X_train_cleaned, y_train_cleaned, verbose=3):
    svr = SVR()
    gs_svr_params = {
    'kernel': ['rbf'],
    'C': np.arange(50, 56, 0.5),
    'epsilon': np.arange(0.1, 0.3, 0.01),
    'gamma' : ['scale', 'auto']
    }
    gs_svr = GridSearchCV(svr, gs_svr_params, cv=5, verbose=verbose)
    gs_svr.fit(X_train_cleaned, y_train_cleaned)
    print(f"""The best validation score obtained is {gs_svr.best_score_:.5f} with
    \tkernel: {gs_svr.best_params_['kernel']}
    \tC: {gs_svr.best_params_['C']}
    \tepsilon: {gs_svr.best_params_['epsilon']}
    \tgamma: {gs_svr.best_params_['gamma']}""")
    return gs_svr

### Model 3: Gradient Boosting

In [125]:
def best_gbr(X_train, y_train):
    gbr = GradientBoostingRegressor()
    gs_gbr_params = {
     "loss":["ls"],
     "learning_rate": [0.03, 0.035, 0.04],
     "n_estimators": np.arange(330, 421, 30),
     "subsample": np.arange(0.3, 0.41, 0.05),
     "max_depth": [4, 8],
     "min_samples_split": [4, 5, 6],
     "min_samples_leaf": [1, 5],
     "random_state": [0], 
     "verbose": [1]
    }
    gs_gbr = GridSearchCV(gbr, gs_gbr_params, cv=4, verbose=3, error_score='raise')
    gs_gbr.fit(X_train, y_train)

    print(f"""The best validation score obtained is {gs_gbr.best_score_:.5f} with
    \tloss: {gs_gbr.best_params_['loss']}
    \tlearning_rate: {gs_gbr.best_params_['learning_rate']}
    \tn_estimators: {gs_gbr.best_params_['n_estimators']}
    \tsubsample: {gs_gbr.best_params_['subsample']}
    \tmax_depth: {gs_gbr.best_params_['max_depth']}
    \tmin_samples_split: {gs_gbr.best_params_['min_samples_split']}
    \tmin_samples_leaf: {gs_gbr.best_params_['min_samples_leaf']}
    """)
        
    return gs_gbr

In [126]:
def old_best_gbr(X_train, y_train):

    params = {
        "loss":"ls",
        "n_estimators": 350,
        "learning_rate": 0.03162277660168379,
        "subsample": 0.4,
        "max_depth": 4,
        "min_samples_split": 5,
        "min_samples_leaf": 1,
        "random_state": 0, 
        "verbose": 1,
    }

    gbr = GradientBoostingRegressor(**params)
    
    # Get the validation score
    gbr_cv_scores = cross_val_score(gbr, X_train, y_train, n_jobs=-1, verbose=3)
    print(f"""Validation score obtained is {np.mean(gbr_cv_scores):.4f} with
    \t{params}""")
    
    # Fit model (because previous function does not return fitted model)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    gbr.fit(X_train, y_train)
    
    test_score = np.zeros(gbr.n_estimators_, dtype=np.float64)
    for i, y_pred in enumerate(gbr.staged_predict(X_test)):
        test_score[i] = gbr.loss_(y_test, y_pred)

    fig = plt.figure(figsize=(6, 6))
    plt.subplot(1, 1, 1)
    plt.title("Deviance")
    plt.plot(
        np.arange(gbr.n_estimators_),
        gbr.train_score_,
        "b-",
        label="Training Set Deviance",
    )
    plt.plot(
        np.arange(gbr.n_estimators_), test_score, "r-", label="Test Set Deviance"
    )
    plt.legend(loc="upper right")
    plt.xlabel("Boosting Iterations")
    plt.ylabel("Deviance")
    fig.tight_layout()
    plt.show()
    
    return gbr

## Model 4: Adaboost

In [127]:
def best_Adaboost(X_train, y_train):
    
    ada = AdaBoostRegressor()
    gs_ada_params = {
     "loss":["square"],
     "learning_rate": np.logspace(-2, -1, 5),
     "n_estimators": np.arange(50, 550, 100),
     "random_state": [0], 
    }
    gs_ada = GridSearchCV(ada, gs_ada_params, cv=5, verbose=3, error_score='raise')
    gs_ada.fit(X_train, y_train)

    print(f"""The best validation score obtained is {gs_ada.best_score_:.5f} with
    \tloss: {gs_ada.best_params_['loss']}
    \tlearning_rate: {gs_ada.best_params_['learning_rate']}
    \tn_estimators: {gs_ada.best_params_['n_estimators']}
    """)
    
    return ada

## Model 5: Random Forest

In [128]:
def best_random_forest(X_train, y_train):
    # define used regressor
    forest=RandomForestRegressor(n_estimators=20,
                                 max_depth=10,
                                 criterion='mse',)

    r_forest = RandomForestRegressor()
    gs_forest_params = {
     "n_estimators": np.arange(50, 500, 50),
     "max_depth": np.arange(2, 8, 1),
     "min_samples_split": np.arange(2, 8, 1),
     "min_samples_leaf": np.arange(1, 9, 2),
     "random_state": [0], 
     "verbose": [1]
    }
    
    gs_forest = GridSearchCV(r_forest, gs_forest_params, cv=5, verbose=3, error_score='raise')
    gs_forest.fit(X_train, y_train)
    

    print(f"""The best validation score obtained is {gs_forest.best_score_:.5f} with
    \tn_estimators: {gs_forest.best_params_['n_estimators']}
    \tmax_depth: {gs_forest.best_params_['max_depth']}
    \tmin_samples_split: {gs_forest.best_params_['min_samples_split']}
    \tmin_samples_leaf: {gs_forest.best_params_['min_samples_leaf']}
    """)
    
    return r_forest

---
## Create CSV for submission

In [129]:
def create_submission(prediction, sub_id, basepath = 'submissions/task1-sub'):
    result = prediction.copy()
    result = result.rename(columns={0: 'y'})
    result['id'] = range(0, len(result))
    result = result[['id', 'y']]
    result.to_csv(basepath+str(sub_id) + '.csv', index=False)

---
## Final Pipeline

In [154]:
def gs_clean(thresholds_too_correlated, ks, max_iters, verbose=0):
    print(f"""Testing all {len(thresholds_too_correlated) * len(ks) * len(max_iters)} possibilities.""")

    general_best_svr=None
    best_k = max_iters[0]
    best_max_iter = ks[0]
    best_threshold = thresholds_too_correlated[0]
    best_X_train = None
    best_y_train = None
    best_X_test = None
    for threshold in thresholds_too_correlated:
        for k in ks:
            for max_iter in max_iters:
                print(f"""\n\nTesting values \n\tk: {k}     threshold: {threshold}     max_iter: {max_iter} """)
                X_train_cleaned, y_train_cleaned, X_test_cleaned = clean_data(threshold, k, max_iter, verbose=verbose)
                current_best_svr = best_svr(X_train_cleaned, np.array(y_train_cleaned).ravel(), verbose=verbose)
                if general_best_svr is None or current_best_svr.best_score_ > general_best_svr.best_score_:
                    general_best_svr = current_best_svr
                    best_k=k
                    best_max_iter=max_iter
                    best_threshold=threshold
                    best_X_train = X_train_cleaned
                    best_y_train = y_train_cleaned
                    best_X_test = X_test_cleaned
                         
    export_to_csv(best_X_train, best_y_train, best_X_test)
               
    print(f"""\n\nThe best validation score obtained is {general_best_svr.best_score_:.5f} 
    \With clean parameters:
    \tk: {best_k}
    \tthreshold: {best_threshold}
    \tmax_iter: {best_max_iter} 
    \nAnd hyperparameters of the svr:
    \tkernel: {general_best_svr.best_params_['kernel']}
    \tC: {general_best_svr.best_params_['C']}
    \tepsilon: {general_best_svr.best_params_['epsilon']}
    \tgamma: {general_best_svr.best_params_['gamma']}
    """)    
    return general_best_svr, best_X_train, best_y_train, best_X_test  

In [139]:
def clean_data(threshold_too_correlated=0.99, k=190, max_iter=45, verbose=2):
    if verbose:
        print("Loading raw data...")
    X_train, y_train, X_test = import_data()
    
    if verbose:
        print("Removing outliers...")
    #X_train_no_outliers, y_train_cleaned = remove_outliers(X_train, y_train, verbose=verbose, method="IsolationForest")
    X_train_no_outliers, y_train_cleaned = remove_outliers(X_train, y_train, verbose=verbose, method="LocalOutlierFactor") #Remove outliers in x

    if verbose:
        print("Scaling data...")
    X_train_scaled, X_test_scaled = scale(X_train_no_outliers, X_test)

    if verbose:
        print("Selecting features...")
    X_train_no_constant_features, X_test_no_constant_features = remove_constant_features(X_train_scaled, X_test_scaled, verbose=verbose)
    X_train_no_too_correlated_features, X_test_no_too_correlated_features = remove_too_coorelated_features(X_train_no_constant_features, X_test_no_constant_features, verbose=verbose, threshold=threshold_too_correlated)

    #TODO: Clean pipeline
    # If we want iterative imputation, and don't want to run it for one hour (should try at one point though), need to
    # do feature selection before we do imputation. Except biggest filter for feature selection is f_regression which
    # cannot deal with nan values. Therefore, intermediary dumb knn imputation was quickly/dirtily 
    # implemented to see results. However, I tried to run Iterative Imputations for just a few iterations and it didn't give
    # better results. Furthermore, we need more than 16GB RAM if we want to run it. Solutions could be to use google colab
    # to access GPU.
    # Conclusion: we achieve ~same or better validation scores with a LOT less features(random features), seems like an 
    # interresting way forward. However, we shouldn't remove too many features with high correlations, cause it makes
    # the score being worse.

    #We first impute with knn which is cheap and then later on reaply mask and use iterative imputing
    X_train_mask = X_train_no_too_correlated_features.isna()
    X_test_mask = X_test_no_too_correlated_features.isna()
    X_train_knn_imputed, X_test_knn_imputed = impute_values(X_train_no_too_correlated_features, X_test_no_too_correlated_features, verbose=verbose, method='knn')
    X_train_selected_features, X_test_selected_features, X_train_mask, X_test_mask = remove_random_features(X_train_knn_imputed, y_train_cleaned, X_test_knn_imputed, X_train_mask, X_test_mask, verbose=verbose, k=k)
    X_train_no_imputation = X_train_selected_features.mask(np.array(X_train_mask))
    X_test_no_imputation = X_test_selected_features.mask(np.array(X_test_mask))

    if verbose:
        print("Imputing nan values...")
    X_train_cleaned, X_test_cleaned = impute_values(X_train_no_imputation, X_test_no_imputation, method='iterative', verbose=verbose, max_iter=max_iter)

    #print("Exporting clean data to csv...")
    #export_to_csv(X_train_cleaned, y_train_cleaned, X_test_cleaned)
    #export_to_csv(X_train_knn_imputed, y_train_cleaned, X_test_knn_imputed)

    print("Data cleaning done!")
    return X_train_cleaned, y_train_cleaned, X_test_cleaned

In [140]:
#gs = model(X_train_cleaned, np.array(y_train_cleaned).ravel())

In [155]:
params = {
    "thresholds_too_correlated":np.arange(0.99, 1, 0.001),
    "ks": np.arange(190, 196, 1),
    "max_iters": [60]
}
gs, X_train_cleaned, y_train_cleaned, X_test_cleaned = gs_clean(**params)

Testing all 66 possibilities.


Testing values 
	k: 190     threshold: 0.99     max_iter: 60 


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


Data cleaning done!
The best validation score obtained is 0.66892 with
    	kernel: rbf
    	C: 55
    	epsilon: 0.1
    	gamma: scale


Testing values 
	k: 191     threshold: 0.99     max_iter: 60 


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


Data cleaning done!
The best validation score obtained is 0.66858 with
    	kernel: rbf
    	C: 55
    	epsilon: 0.15
    	gamma: auto


Testing values 
	k: 192     threshold: 0.99     max_iter: 60 


...

Data cleaning done!
The best validation score obtained is 0.66554 with
    	kernel: rbf
    	C: 63
    	epsilon: 0.2
    	gamma: scale


Testing values 
	k: 194     threshold: 1.0     max_iter: 60 


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


Data cleaning done!
The best validation score obtained is 0.66527 with
    	kernel: rbf
    	C: 63
    	epsilon: 0.2
    	gamma: scale


Testing values 
	k: 195     threshold: 1.0     max_iter: 60 


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


Data cleaning done!
The best validation score obtained is 0.66482 with
    	kernel: rbf
    	C: 63
    	epsilon: 0.25
    	gamma: scale


The best validation score obtained is 0.67188 
    \With clean parameters:
    	k: 193
    	threshold: 0.994
    	max_iter: 60 
    
And hyperparameters of the svr:
    	kernel: rbf
    	C: 55
    	epsilon: 0.2
    	gamma: scale
    


In [35]:
gs_gbr = best_gbr(X_train_cleaned, np.array(y_train_cleaned).ravel())

Fitting 4 folds for each of 432 candidates, totalling 1728 fits
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          79.5440           2.6846            3.79s
         2          87.5767           2.6863            3.82s
         3          77.5899           2.1031            3.88s
         4          82.5057           2.0087            3.89s
         5          77.5680           1.9622            3.88s
         6          81.1205           2.1568            3.84s
         7          75.8315           1.5874            3.83s
         8          79.1193           1.8987            3.80s
         9          74.0108           1.6469            3.78s
        10          64.2041           1.2798            3.79s
        20          55.0562           1.0745            3.61s
        30          43.8678           0.6181            3.44s
        40          36.5444           0.3009            3.30s
        50          30.7234           0.1529            3.16s
     

        20          58.8633           0.9136            4.23s
        30          42.6490           0.6546            4.10s
        40          35.4246           0.3389            3.94s
        50          28.3536           0.2020            3.78s
        60          28.0710           0.1298            3.64s
        70          25.3890           0.1214            3.49s
        80          25.0154           0.0074            3.38s
        90          18.6709           0.0703            3.27s
       100          20.3632          -0.0449            3.17s
       200          11.2848          -0.0141            1.86s
       300           5.6976          -0.0058            0.44s
[CV 2/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=330, random_state=0, subsample=0.35, verbose=1;, score=0.639 total time=   4.8s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          96.2257           2.3911            4.60s
    

       100          16.1144           0.0825            3.48s
       200           7.0128          -0.0059            2.02s
       300           4.0501          -0.0016            0.46s
[CV 3/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=330, random_state=0, subsample=0.39999999999999997, verbose=1;, score=0.585 total time=   5.1s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          97.2166           2.5536            4.87s
         2          84.3763           2.5687            4.86s
         3          85.7451           2.1271            4.85s
         4          74.6708           1.9484            4.87s
         5          76.1027           1.5691            4.84s
         6          74.7466           1.6727            4.81s
         7          70.3133           1.9009            4.78s
         8          65.2464           1.7358            4.80s
         9          64.5661           1.4237       

        20          52.9221           0.9989            4.41s
        30          43.3489           0.6317            4.33s
        40          35.2305           0.2527            4.23s
        50          30.6584           0.2052            4.08s
        60          27.3894           0.0865            3.95s
        70          25.2602           0.1167            3.82s
        80          24.2942           0.0277            3.73s
        90          19.1922           0.0223            3.61s
       100          20.7587           0.0444            3.49s
       200          11.1155          -0.0255            2.22s
       300           5.5340          -0.0156            0.83s
[CV 1/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=360, random_state=0, subsample=0.35, verbose=1;, score=0.651 total time=   4.9s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          95.1973           2.4750            5.75s
    

       100          20.0888           0.0191            4.22s
       200          10.6169          -0.0204            2.56s
       300           5.2060          -0.0041            0.95s
[CV 2/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=360, random_state=0, subsample=0.39999999999999997, verbose=1;, score=0.647 total time=   5.7s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          95.3174           2.5721            5.27s
         2          81.3937           2.4040            5.36s
         3          81.5316           2.2148            5.26s
         4          78.4736           1.8265            5.26s
         5          72.7746           2.0110            5.30s
         6          70.9764           1.9588            5.31s
         7          67.7876           1.7316            5.34s
         8          65.3826           1.7840            5.32s
         9          64.5150           1.4585       

        20          56.3656           0.9862            4.49s
        30          43.5765           0.5589            4.45s
        40          32.6860           0.3403            4.29s
        50          27.5054           0.1692            4.18s
        60          25.5135           0.1702            4.09s
        70          24.4987           0.0405            3.95s
        80          23.9978           0.0224            3.84s
        90          20.7755           0.0121            3.75s
       100          19.3132           0.0314            3.63s
       200           9.9404          -0.0196            2.46s
       300           7.2797          -0.0154            1.16s
[CV 4/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=390, random_state=0, subsample=0.3, verbose=1;, score=0.657 total time=   5.0s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          79.3263           2.5131            6.09s
     

       100          19.5630           0.0551            4.65s
       200          10.1560           0.0122            3.04s
       300           5.1777          -0.0065            1.46s
[CV 1/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=390, random_state=0, subsample=0.39999999999999997, verbose=1;, score=0.658 total time=   6.4s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          92.9124           2.5373            5.62s
         2          93.2745           2.4676            5.76s
         3          86.2939           1.8576            5.79s
         4          85.4408           2.3074            5.75s
         5          82.2286           2.1695            5.73s
         6          80.6460           2.0977            5.71s
         7          79.0957           1.7879            5.71s
         8          78.9759           1.8673            5.69s
         9          80.6011           1.2838       

        20          59.1763           1.0561            4.87s
        30          38.7821           0.5528            4.71s
        40          33.5819           0.3505            4.61s
        50          26.6965           0.2133            4.48s
        60          24.3530           0.1418            4.40s
        70          20.4838           0.0155            4.29s
        80          24.0969           0.1715            4.14s
        90          18.2038           0.0415            3.98s
       100          16.5148          -0.0490            3.84s
       200           7.7988          -0.0054            2.75s
       300           5.4578          -0.0102            1.56s
       400           3.3339          -0.0036            0.25s
[CV 3/4] END learning_rate=0.03, loss=ls, max_depth=4, min_samples_leaf=1, min_samples_split=4, n_estimators=420, random_state=0, subsample=0.3, verbose=1;, score=0.578 total time=   5.3s
      Iter       Train Loss      OOB Improve   Remaining Time 
     

KeyboardInterrupt: 

In [158]:
X_train_cleaned, y_train_cleaned, X_test_cleaned = clean_data(0.994, 193, 53)

Loading raw data...
Removing outliers...
Detected 50 outliers with method LocalOutlierFactor, out of 1212 samples (4.13%).
Scaling data...
Selecting features...
3 features removed because of constant values (0.36%)
24 features removed because of correlation > 0.994 (2.90%)
For the train dataset, there are 71288 nan values, out of 935410 (7.62%).


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms


612 features removed because of low correlation with target (76.02%).
Imputing nan values...
For the train dataset, there are 17034 nan values, out of 224266 (7.60%).
[IterativeImputer] Completing matrix with shape (1162, 193)
[IterativeImputer] Ending imputation round 1/53, elapsed time 2.40
[IterativeImputer] Change: 46.32551823189247, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 2/53, elapsed time 4.88
[IterativeImputer] Change: 6.109456462424729, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 3/53, elapsed time 7.41
[IterativeImputer] Change: 2.2677467588632836, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 4/53, elapsed time 10.04
[IterativeImputer] Change: 0.6677910194206041, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 5/53, elapsed time 13.29
[IterativeImputer] Change: 0.3912612071554267, scaled tolerance: 0.014114220441223293 
[It

[IterativeImputer] Ending imputation round 52/53, elapsed time 176.59
[IterativeImputer] Change: 0.034787685142495445, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Ending imputation round 53/53, elapsed time 179.87
[IterativeImputer] Change: 0.027486891542934544, scaled tolerance: 0.014114220441223293 
[IterativeImputer] Completing matrix with shape (1162, 193)
[IterativeImputer] Ending imputation round 1/53, elapsed time 0.07
[IterativeImputer] Ending imputation round 2/53, elapsed time 0.14


/home/loic/Enter/envs/InternetAnalytics/lib/python3.9/site-packages/sklearn/impute/_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


[IterativeImputer] Ending imputation round 3/53, elapsed time 0.22
[IterativeImputer] Ending imputation round 4/53, elapsed time 0.29
[IterativeImputer] Ending imputation round 5/53, elapsed time 0.35
[IterativeImputer] Ending imputation round 6/53, elapsed time 0.42
[IterativeImputer] Ending imputation round 7/53, elapsed time 0.49
[IterativeImputer] Ending imputation round 8/53, elapsed time 0.56
[IterativeImputer] Ending imputation round 9/53, elapsed time 0.62
[IterativeImputer] Ending imputation round 10/53, elapsed time 0.69
[IterativeImputer] Ending imputation round 11/53, elapsed time 0.76
[IterativeImputer] Ending imputation round 12/53, elapsed time 0.82
[IterativeImputer] Ending imputation round 13/53, elapsed time 0.89
[IterativeImputer] Ending imputation round 14/53, elapsed time 0.96
[IterativeImputer] Ending imputation round 15/53, elapsed time 1.02
[IterativeImputer] Ending imputation round 16/53, elapsed time 1.09
[IterativeImputer] Ending imputation round 17/53, elaps

In [166]:
gs = best_svr(X_train_cleaned, np.array(y_train_cleaned).ravel())

Fitting 5 folds for each of 480 candidates, totalling 2400 fits
[CV 1/5] END C=50.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.760 total time=   0.2s
[CV 2/5] END C=50.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=50.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=50.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.672 total time=   0.2s
[CV 5/5] END C=50.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.675 total time=   0.2s
[CV 1/5] END C=50.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.761 total time=   0.2s
[CV 2/5] END C=50.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.675 total time=   0.2s
[CV 3/5] END C=50.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.575 total time=   0.2s
[CV 4/5] END C=50.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.672 total time=   0.2s
[CV 5/5] END C=50.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=50.0, epsilon=0.

[CV 4/5] END C=50.0, epsilon=0.17999999999999997, gamma=scale, kernel=rbf;, score=0.671 total time=   0.2s
[CV 5/5] END C=50.0, epsilon=0.17999999999999997, gamma=scale, kernel=rbf;, score=0.675 total time=   0.2s
[CV 1/5] END C=50.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.762 total time=   0.2s
[CV 2/5] END C=50.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=50.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=50.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.671 total time=   0.2s
[CV 5/5] END C=50.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=50.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.760 total time=   0.2s
[CV 2/5] END C=50.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=50.0, epsilon=0.18999999999

[CV 2/5] END C=50.0, epsilon=0.2599999999999999, gamma=scale, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=50.0, epsilon=0.2599999999999999, gamma=scale, kernel=rbf;, score=0.577 total time=   0.2s
[CV 4/5] END C=50.0, epsilon=0.2599999999999999, gamma=scale, kernel=rbf;, score=0.671 total time=   0.2s
[CV 5/5] END C=50.0, epsilon=0.2599999999999999, gamma=scale, kernel=rbf;, score=0.674 total time=   0.2s
[CV 1/5] END C=50.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.762 total time=   0.2s
[CV 2/5] END C=50.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=50.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=50.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.670 total time=   0.2s
[CV 5/5] END C=50.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=50.0, epsilon=0.2699999999999999, ga

[CV 2/5] END C=50.5, epsilon=0.13999999999999999, gamma=auto, kernel=rbf;, score=0.675 total time=   0.2s
[CV 3/5] END C=50.5, epsilon=0.13999999999999999, gamma=auto, kernel=rbf;, score=0.575 total time=   0.2s
[CV 4/5] END C=50.5, epsilon=0.13999999999999999, gamma=auto, kernel=rbf;, score=0.672 total time=   0.2s
[CV 5/5] END C=50.5, epsilon=0.13999999999999999, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=50.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.760 total time=   0.2s
[CV 2/5] END C=50.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=50.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.577 total time=   0.2s
[CV 4/5] END C=50.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.672 total time=   0.2s
[CV 5/5] END C=50.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.675 total time=   0.2s
[CV 1/5] END C=50.5, epsilon=0.1499999999

[CV 4/5] END C=50.5, epsilon=0.21999999999999995, gamma=scale, kernel=rbf;, score=0.671 total time=   0.2s
[CV 5/5] END C=50.5, epsilon=0.21999999999999995, gamma=scale, kernel=rbf;, score=0.674 total time=   0.2s
[CV 1/5] END C=50.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.762 total time=   0.2s
[CV 2/5] END C=50.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=50.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=50.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.671 total time=   0.2s
[CV 5/5] END C=50.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=50.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.760 total time=   0.2s
[CV 2/5] END C=50.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=50.5, epsilon=0.22999999999

...

[CV 4/5] END C=55.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.673 total time=   0.2s
[CV 5/5] END C=55.0, epsilon=0.1, gamma=scale, kernel=rbf;, score=0.675 total time=   0.2s
[CV 1/5] END C=55.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.761 total time=   0.2s
[CV 2/5] END C=55.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.675 total time=   0.2s
[CV 3/5] END C=55.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.574 total time=   0.2s
[CV 4/5] END C=55.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.674 total time=   0.2s
[CV 5/5] END C=55.0, epsilon=0.1, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=55.0, epsilon=0.11, gamma=scale, kernel=rbf;, score=0.759 total time=   0.2s
[CV 2/5] END C=55.0, epsilon=0.11, gamma=scale, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=55.0, epsilon=0.11, gamma=scale, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=55.0, epsilon=0.11, gamma=scale, kernel=rbf;, score=0.673 total time=   0.2s


[CV 2/5] END C=55.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=55.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.575 total time=   0.2s
[CV 4/5] END C=55.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 5/5] END C=55.0, epsilon=0.17999999999999997, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=55.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.759 total time=   0.2s
[CV 2/5] END C=55.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=55.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=55.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.673 total time=   0.2s
[CV 5/5] END C=55.0, epsilon=0.18999999999999995, gamma=scale, kernel=rbf;, score=0.675 total time=   0.2s
[CV 1/5] END C=55.0, epsilon=0.1899999999

[CV 5/5] END C=55.0, epsilon=0.2599999999999999, gamma=scale, kernel=rbf;, score=0.674 total time=   0.2s
[CV 1/5] END C=55.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.762 total time=   0.2s
[CV 2/5] END C=55.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=55.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.575 total time=   0.2s
[CV 4/5] END C=55.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.672 total time=   0.2s
[CV 5/5] END C=55.0, epsilon=0.2599999999999999, gamma=auto, kernel=rbf;, score=0.672 total time=   0.2s
[CV 1/5] END C=55.0, epsilon=0.2699999999999999, gamma=scale, kernel=rbf;, score=0.759 total time=   0.2s
[CV 2/5] END C=55.0, epsilon=0.2699999999999999, gamma=scale, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=55.0, epsilon=0.2699999999999999, gamma=scale, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=55.0, epsilon=0.2699999999999999, ga

[CV 4/5] END C=55.5, epsilon=0.13999999999999999, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 5/5] END C=55.5, epsilon=0.13999999999999999, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=55.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.759 total time=   0.2s
[CV 2/5] END C=55.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=55.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=55.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.673 total time=   0.2s
[CV 5/5] END C=55.5, epsilon=0.14999999999999997, gamma=scale, kernel=rbf;, score=0.675 total time=   0.2s
[CV 1/5] END C=55.5, epsilon=0.14999999999999997, gamma=auto, kernel=rbf;, score=0.761 total time=   0.2s
[CV 2/5] END C=55.5, epsilon=0.14999999999999997, gamma=auto, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=55.5, epsilon=0.1499999999

[CV 2/5] END C=55.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.676 total time=   0.2s
[CV 3/5] END C=55.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.575 total time=   0.2s
[CV 4/5] END C=55.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 5/5] END C=55.5, epsilon=0.21999999999999995, gamma=auto, kernel=rbf;, score=0.673 total time=   0.2s
[CV 1/5] END C=55.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.759 total time=   0.2s
[CV 2/5] END C=55.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.677 total time=   0.2s
[CV 3/5] END C=55.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.576 total time=   0.2s
[CV 4/5] END C=55.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.672 total time=   0.2s
[CV 5/5] END C=55.5, epsilon=0.22999999999999995, gamma=scale, kernel=rbf;, score=0.674 total time=   0.2s
[CV 1/5] END C=55.5, epsilon=0.2299999999

In [180]:
r2_score(y_train_cleaned, np.ceil(gs.predict(X_train_cleaned)))

0.960772007094017

In [179]:
prediction = pd.DataFrame(np.ceil(gs.predict(X_test_cleaned)))
SUB_ID = 30 #to modify
create_submission(prediction, SUB_ID)